# Getting Started with Cabruca Segmentation

This notebook provides a comprehensive introduction to the Cabruca Segmentation system, covering basic usage, visualization, and analysis workflows.

## Contents
1. Environment Setup
2. Loading Pre-trained Models
3. Running Inference on Images
4. Visualizing Results
5. Calculating Agroforestry Metrics
6. Exporting Results

## 1. Environment Setup

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path
import json
import pandas as pd
from IPython.display import Image, display

# Add project source to path
sys.path.append('../src')

# Import project modules
from inference.batch_inference import BatchInferenceEngine, VisualizationTools
from evaluation.agroforestry_metrics import AgroforestryMetrics
from integration.theobroma_integration import TheobromaIntegration

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

print(f"PyTorch version: {torch.__version__}")
print(f"MPS available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else False}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Loading Pre-trained Models

In [ ]:
# Model configuration
MODEL_PATH = "../outputs/checkpoint_best.pth"
DEVICE = "auto"  # Will automatically select best available device

# Check if model exists
if not Path(MODEL_PATH).exists():
    print(f"⚠️ Model not found at {MODEL_PATH}")
    print("Please train a model first or download a pre-trained checkpoint")
else:
    # Initialize inference engine
    print("Loading model...")
    engine = BatchInferenceEngine(
        model_path=MODEL_PATH,
        device=DEVICE
    )
    print(f"✅ Model loaded successfully on {engine.device}")

## 3. Running Inference on Images

In [ ]:
# Load a sample image
# You can replace this with your own image path
sample_image_path = "../data/test/sample_plantation.jpg"

# Create a sample image if it doesn't exist
if not Path(sample_image_path).exists():
    print("Creating sample image...")
    os.makedirs(Path(sample_image_path).parent, exist_ok=True)
    
    # Create synthetic plantation image
    import cv2
    img = np.zeros((512, 512, 3), dtype=np.uint8)
    img[:, :] = [34, 139, 34]  # Forest green background
    
    # Add tree-like circles
    np.random.seed(42)
    for _ in range(20):
        x, y = np.random.randint(50, 462, 2)
        radius = np.random.randint(15, 30)
        color = [0, 100, 0] if np.random.random() > 0.3 else [139, 69, 19]
        cv2.circle(img, (x, y), radius, color, -1)
    
    cv2.imwrite(sample_image_path, img)
    print(f"Sample image created at {sample_image_path}")

# Run inference
print("\nRunning inference...")
result = engine.process_single(sample_image_path)

# Display results summary
print(f"\n📊 Inference Results:")
print(f"  Trees detected: {len(result.trees)}")
print(f"  - Cacao trees: {sum(1 for t in result.trees if t.species == 'cacao')}")
print(f"  - Shade trees: {sum(1 for t in result.trees if t.species == 'shade')}")
print(f"  Canopy density: {result.canopy_density:.2%}")
print(f"  Processing time: {result.processing_time:.2f}s")

## 4. Visualizing Results

In [ ]:
# Load original image
import cv2
image = cv2.imread(sample_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Create visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Original image
axes[0, 0].imshow(image)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

# 2. Detection overlay
overlay = VisualizationTools.create_overlay(image, result, alpha=0.4)
axes[0, 1].imshow(overlay)
axes[0, 1].set_title('Detection Overlay')
axes[0, 1].axis('off')

# 3. Semantic segmentation
semantic_colored = BatchInferenceEngine.SEMANTIC_COLORS[result.semantic_map]
axes[0, 2].imshow(semantic_colored.astype(np.uint8))
axes[0, 2].set_title('Semantic Segmentation')
axes[0, 2].axis('off')

# 4. Tree density heatmap
density_heatmap = VisualizationTools.create_heatmap(result, 'density')
axes[1, 0].imshow(density_heatmap)
axes[1, 0].set_title('Tree Density Heatmap')
axes[1, 0].axis('off')

# 5. Crown diameter heatmap
crown_heatmap = VisualizationTools.create_heatmap(result, 'crown')
axes[1, 1].imshow(crown_heatmap)
axes[1, 1].set_title('Crown Diameter Heatmap')
axes[1, 1].axis('off')

# 6. Species distribution
species_heatmap = VisualizationTools.create_heatmap(result, 'species')
axes[1, 2].imshow(species_heatmap)
axes[1, 2].set_title('Species Distribution')
axes[1, 2].axis('off')

plt.suptitle('Cabruca Segmentation Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Tree inventory table
if result.trees:
    tree_data = []
    for tree in result.trees[:10]:  # Show first 10 trees
        tree_data.append({
            'ID': tree.id,
            'Species': tree.species,
            'Confidence': f"{tree.confidence:.2f}",
            'Crown Diameter (m)': f"{tree.crown_diameter:.2f}",
            'Crown Area (m²)': f"{tree.crown_area:.2f}",
            'X': f"{tree.centroid[0]:.1f}",
            'Y': f"{tree.centroid[1]:.1f}"
        })
    
    df = pd.DataFrame(tree_data)
    print("\n🌳 Tree Inventory (First 10 trees):")
    display(df)

## 5. Calculating Agroforestry Metrics

In [ ]:
# Initialize metrics calculator
metrics_calculator = AgroforestryMetrics()

# Convert result to format expected by metrics calculator
predictions = {
    'boxes': [],
    'labels': [],
    'scores': [],
    'masks': [],
    'semantic_map': result.semantic_map
}

for tree in result.trees:
    # Create bounding box from centroid and crown diameter
    cx, cy = tree.centroid
    r = tree.crown_diameter / 2
    box = [cx - r, cy - r, cx + r, cy + r]
    
    predictions['boxes'].append(box)
    predictions['labels'].append(1 if tree.species == 'cacao' else 2)
    predictions['scores'].append(tree.confidence)

# Calculate metrics
metrics = metrics_calculator.calculate_all_metrics(predictions)

# Display metrics
print("📊 Agroforestry Metrics:")
print("\n1. Tree Detection:")
print(f"   Total trees: {metrics['tree_count']['total']}")
print(f"   Cacao trees: {metrics['tree_count']['cacao']}")
print(f"   Shade trees: {metrics['tree_count']['shade']}")

print("\n2. Canopy Coverage:")
print(f"   Total coverage: {metrics['canopy_coverage']['total_coverage']:.2%}")
print(f"   Cacao coverage: {metrics['canopy_coverage']['cacao_coverage']:.2%}")
print(f"   Shade coverage: {metrics['canopy_coverage']['shade_coverage']:.2%}")

print("\n3. Land Cover Distribution:")
for class_name, percentage in metrics['land_cover'].items():
    print(f"   {class_name}: {percentage:.2%}")

print("\n4. Spatial Distribution:")
print(f"   Clark-Evans Index: {metrics['spacing']['clark_evans_index']:.2f}")
print(f"   Pattern: {metrics['spacing']['pattern']}")

print("\n5. Crown Statistics:")
print(f"   Mean crown diameter: {metrics['crown_statistics']['mean_diameter']:.2f}m")
print(f"   Std crown diameter: {metrics['crown_statistics']['std_diameter']:.2f}m")

## 6. Plantation Health Assessment

In [ ]:
# Generate health assessment
def assess_plantation_health(metrics):
    """Generate health assessment based on metrics."""
    health_score = 0
    max_score = 100
    recommendations = []
    
    # 1. Shade-to-cacao ratio (optimal: 1:5)
    if metrics['tree_count']['cacao'] > 0:
        shade_ratio = metrics['tree_count']['shade'] / metrics['tree_count']['cacao']
        optimal_ratio = 0.2
        
        if 0.15 <= shade_ratio <= 0.25:
            health_score += 25
        elif 0.1 <= shade_ratio <= 0.3:
            health_score += 15
        else:
            health_score += 5
            
        if shade_ratio < 0.15:
            recommendations.append("Plant additional shade trees for optimal cacao growth")
        elif shade_ratio > 0.25:
            recommendations.append("Consider removing some shade trees to increase cacao productivity")
    
    # 2. Canopy density (optimal: 35-55%)
    canopy_density = metrics['canopy_coverage']['total_coverage']
    if 0.35 <= canopy_density <= 0.55:
        health_score += 25
    elif 0.25 <= canopy_density <= 0.65:
        health_score += 15
    else:
        health_score += 5
        
    if canopy_density < 0.3:
        recommendations.append("Low canopy density detected. Consider planting more trees")
    elif canopy_density > 0.6:
        recommendations.append("High canopy density may reduce productivity. Consider selective pruning")
    
    # 3. Spatial distribution
    clark_evans = metrics['spacing']['clark_evans_index']
    if 0.8 <= clark_evans <= 1.2:
        health_score += 25
    elif 0.6 <= clark_evans <= 1.4:
        health_score += 15
    else:
        health_score += 5
        
    if clark_evans < 0.8:
        recommendations.append("Trees are clustered. Consider redistributing for better resource utilization")
    elif clark_evans > 1.2:
        recommendations.append("Trees are over-dispersed. Fill gaps to optimize land use")
    
    # 4. Understory management
    understory = metrics['land_cover'].get('understory', 0)
    if 0.2 <= understory <= 0.4:
        health_score += 25
    elif 0.1 <= understory <= 0.5:
        health_score += 15
    else:
        health_score += 5
        
    if understory < 0.2:
        recommendations.append("Low understory coverage. Consider cover crops to prevent erosion")
    
    # Determine overall status
    if health_score >= 80:
        status = "Excellent"
    elif health_score >= 60:
        status = "Good"
    elif health_score >= 40:
        status = "Moderate"
    else:
        status = "Poor"
    
    return {
        'score': health_score,
        'max_score': max_score,
        'percentage': health_score / max_score,
        'status': status,
        'recommendations': recommendations
    }

# Generate health assessment
health_assessment = assess_plantation_health(metrics)

print("🏥 Plantation Health Assessment")
print("=" * 40)
print(f"Overall Score: {health_assessment['score']}/{health_assessment['max_score']} ({health_assessment['percentage']:.0%})")
print(f"Status: {health_assessment['status']}")

if health_assessment['recommendations']:
    print("\n📋 Recommendations:")
    for i, rec in enumerate(health_assessment['recommendations'], 1):
        print(f"  {i}. {rec}")

# Visualize health score
fig, ax = plt.subplots(figsize=(8, 6))
categories = ['Shade Ratio', 'Canopy Density', 'Spatial Distribution', 'Understory']
scores = [25, 25, 25, 25]  # Placeholder scores for visualization

bars = ax.bar(categories, scores, color=['green' if s >= 20 else 'orange' if s >= 10 else 'red' for s in scores])
ax.set_ylabel('Score')
ax.set_title('Plantation Health Components')
ax.set_ylim(0, 30)

for bar, score in zip(bars, scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{score}/25', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 7. Exporting Results

In [ ]:
# Export to GeoJSON
output_dir = Path("../outputs/notebook_results")
output_dir.mkdir(parents=True, exist_ok=True)

# 1. Export tree inventory to CSV
if result.trees:
    tree_df = pd.DataFrame([
        {
            'id': tree.id,
            'species': tree.species,
            'confidence': tree.confidence,
            'x': tree.centroid[0],
            'y': tree.centroid[1],
            'crown_diameter': tree.crown_diameter,
            'crown_area': tree.crown_area
        }
        for tree in result.trees
    ])
    
    csv_path = output_dir / "tree_inventory.csv"
    tree_df.to_csv(csv_path, index=False)
    print(f"✅ Tree inventory saved to {csv_path}")

# 2. Export to GeoJSON
from inference.batch_inference import ReportGenerator

geojson_path = output_dir / "trees.geojson"
ReportGenerator.export_to_geojson(result, str(geojson_path))
print(f"✅ GeoJSON saved to {geojson_path}")

# 3. Save metrics report
metrics_report = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'image_path': sample_image_path,
    'metrics': metrics,
    'health_assessment': health_assessment
}

json_path = output_dir / "analysis_report.json"
with open(json_path, 'w') as f:
    json.dump(metrics_report, f, indent=2, default=str)
print(f"✅ Analysis report saved to {json_path}")

# 4. Save visualization
viz_path = output_dir / "visualization.png"
fig = VisualizationTools.create_comparison_figure(image, result, str(viz_path))
print(f"✅ Visualization saved to {viz_path}")

print(f"\n📂 All results saved to {output_dir}")

## 8. Next Steps

Congratulations! You've completed the basic workflow. Here are some next steps:

1. **Train a Custom Model**: See `02_training_custom_model.ipynb`
2. **Advanced Analysis**: See `03_analysis_workflow.ipynb`
3. **API Integration**: Use the REST API for production deployment
4. **QGIS Integration**: Load GeoJSON results in QGIS for spatial analysis
5. **Batch Processing**: Process multiple images using the batch inference pipeline

### Resources
- [API Documentation](../API_DOCUMENTATION.md)
- [Training Guide](../docs/training_guide.md)
- [Best Practices](../docs/best_practices.md)